In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import datetime
import os
from skimage.transform import resize
import random
from tensorflow.python.keras.utils.data_utils import Sequence
from scipy.ndimage import zoom
from scipy.ndimage import shift
from skimage.transform import rotate
from sklearn import metrics

In [2]:
#input variables
target_path = "C:\\Users\\awe2\\DL_DES-master\\data\\sdss-galaxyzoo\\high_certainty\\sdss_metadata\\"

#path = 'image_arrays_new_new\\'
#validation_path = path + 'validation'
#training_path = path + 'training'
#test_path = path + 'test'

#model variables
batch_size = 32 #
#epoch_number = 50
learning_rate = 1e-4 

params = {'dim': (120,120),
          'batch_size': batch_size,
          'n_classes': 2,
          'n_channels': 5,
          'shuffle': True}


#more parameters means more prone to overfitting, and I am 5/3 times worse on parameters compared to the paper I have
#based this on. (5 bands instead of 3) I need to find ways to add more regularization, or otherwise might try reducing my number
#of layers to reduce the number of parameters.

In [3]:
#https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions/48097478
def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

In [4]:
#https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
class DataGenerator(Sequence):

    def __init__(self, list_IDs, labels, batch_size=32, dim=(120,120), n_channels=3,
                 n_classes=2, shuffle=True):
     #   'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
    #'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
    #'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
    

      # Generate data and perform augmentation
        for i, ID in enumerate(list_IDs_temp):
            
          # Store sample
            X[i,] = np.load('HP_inputs/' + ID + '.npy')
            
            #I messed up when generating my inputs on the order of the band; I want to train the model in the usual bands:
            
            #u_filter = X[i,:,:,0]
            #g_filter = X[i,:,:,1]
            i_filter = X[i,:,:,2]
            r_filter = X[i,:,:,3]
            #z_filter = X[i,:,:,4]

            #X[i,:,:,0] = u_filter
            #X[i,:,:,1] = g_filter
            X[i,:,:,2] = r_filter
            X[i,:,:,3] = i_filter
            #X[i,:,:,4] = z_filter              
            #flip
            if random.random() > 0.5:
                X[i,] = np.flip(X[i,],0)
            if random.random() > 0.5:
                X[i,] = np.flip(X[i,],1)
            
            #shift
            if random.random() > 0.5 :
                X[i,] = shift(X[i,], (4,0,0), mode='nearest')
            elif random.random() > 0.5 :
                X[i,] = shift(X[i,], (-4,0,0), mode='nearest')
                              
            if random.random() > 0.5 :
                X[i,] = shift(X[i,], (0,4,0), mode='nearest')
            elif random.random() > 0.5 :
                X[i,] = shift(X[i,], (0,-4,0), mode='nearest')
          
            #zoom in/out
            zoom_factor = random.uniform(0.75,1.3)
            X[i,] = clipped_zoom(X[i,],zoom_factor)
            
            #rotate
            angle = 45*random.random()
            X[i,] = rotate(X[i,], angle=angle, mode='reflect')
            
            # Store class
            y[i] = self.labels[ID]
    
        if self.n_classes > 2:
            return X, keras.utils.to_categorical(y, num_classes=self.n_classes)
        else:
            return X, y

    def __len__(self):
    #'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
    #  'Generate one batch of data'
      # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

      # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

      # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

In [5]:
#generate the targets.
a = []
for file in (os.listdir(target_path)):
    target = pd.read_csv(target_path+file,usecols=[14])
    a.append((target["target"].values))
targets = np.concatenate([a[0],a[1],a[2],a[3],a[4],a[5],a[6],a[7],a[8]])

In [6]:
#targets each correspond to an array called: HP_*****.npy in folder HP_inputs// where ***** is zfilled starting at 00000
#we can use os.listdir to generate strings of the correct iterator

#we then need to sort out where each of these files will go.
#for i,file in enumerate(os.listdir("HP_inputs"))

"""
labels = {}
for i in range(len(targets)):
    string = "HP_{}.npy".format(str(i).zfill(5))
    if string in os.listdir("HP_inputs"):
        name = "HP_{}".format(str(i).zfill(5))
        labels.update({name:targets[i]})
        
f = open("HP_inputs\\labels.txt","w")
f.write( str(labels) )
f.close()
"""
with open("labels.txt","r") as inf:
    labels = eval(inf.read())

In [7]:
#now we shuffle our data list, and partition into train, test, and validation sets.
data_list = os.listdir("HP_inputs")
random.shuffle(data_list)
#we want 20% of the data reserved for testing, and then another 15% reserved for validation.
top_index = len(data_list)
first_index = int(round(top_index * 0.65))
second_index = int(round(top_index * 0.15)) + first_index

train_list = data_list[0:first_index]
val_list = data_list[first_index:second_index]
test_list = data_list[second_index::]

for i,file in enumerate(train_list):
    train_list[i] = file.split('.')[0]
for i,file in enumerate(val_list):
    val_list[i] = file.split('.')[0]
for i,file in enumerate(test_list):
    test_list[i] = file.split('.')[0]

partition = {'train':train_list,'validation':val_list,'test':test_list}

In [8]:
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)
test_generator = DataGenerator(partition['test'], labels, **params)

In [9]:
#so this is pretty neat, you can create a keras callback to display on tensorboard using a simplified summary tf api

#and also this is an example of how to change the lr on the fly, which is pretty handy
#https://keras.io/callbacks/


"""
    file_writer = tf.summary.create_file_writer(logdir + "/metrics")
    file_writer.set_as_default()
"""
def lr_schedule(epoch,lr):

#Returns a custom learning rate that decreases as epochs progress.
    if epoch > 1:
        lr = 1e-4
    if epoch > 7:
        lr = 1e-5
    if epoch > 12:
        lr = 1e-6
    if epoch > 20:
        lr = 5e-6

    tf.summary.scalar('learning_rate', tensor=lr)
    return lr

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)

#logdir="summaries/scalars/" + str(datetime.datetime.now().timestamp())
#tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir,
#                                                   histogram_freq=1,
#                                                   write_graph=False,
#                                                   write_grads=True,)
#                                                   #write_images=True)
#will it still print stuff

In [23]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    keras.backend.get_session().run(tf.local_variables_initializer())
    return auc

In [24]:
def create_model(learning_rate=learning_rate):
    
    model = keras.Sequential([])
    
    model.add(keras.layers.Conv2D(input_shape=(120,120,5),filters=32,kernel_size=6,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=0.5))
    
    model.add(keras.layers.Conv2D(filters=64,kernel_size=5,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=2,))
    model.add(keras.layers.Dropout(rate=0.25)) #best = 0.25
    
    model.add(keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=2,))
    model.add(keras.layers.Dropout(rate=0.25)) #best = 0.25
    
    
    model.add(keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=0.35)) #best = 0.35

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=64,activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(units=1,activation=tf.nn.sigmoid)) #tf.nn.softmax for categorical, sigmoid for binary
    
    adam = keras.optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy',auc]) 
    return(model)

In [25]:
keras.backend.clear_session()
model = create_model(learning_rate = learning_rate)

In [12]:
steps_to_take = int(len(train_list)/batch_size)
val_steps_to_take = int(len(val_list)/batch_size)
test_steps_to_take = int(len(test_list)/batch_size)
                #typically be equal to the number of unique samples if your dataset
                #divided by the batch size.

print(steps_to_take)
print(val_steps_to_take)
print(test_steps_to_take)

785
181
241


In [28]:
#another callback
#reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-8, verbose=1, mode='min')

In [29]:
filepath = "models//5-band-CNN-correct-filters.hdf5"
ModelCheckpointCB = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

In [30]:
hist = model.fit_generator(generator=training_generator,
                    steps_per_epoch=steps_to_take, 
                    epochs=20,
                    validation_data=validation_generator,
                    validation_steps=val_steps_to_take,
                    verbose=1,
                    callbacks=[ModelCheckpointCB,lr_callback])
                    #callbacks=[tensorboard_callback,lr_callback])

Epoch 1/20


C:\Users\awe2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


784/785 [============================>.] - ETA: 2s - loss: 1.0354 - acc: 0.5412
Epoch 00001: val_loss improved from inf to 0.64114, saving model to models//5-band-CNN-correct-filters.hdf5
785/785 [==============================] - 2708s 3s/step - loss: 1.0349 - acc: 0.5414 - val_loss: 0.6411 - val_acc: 0.5478
Epoch 2/20
784/785 [============================>.] - ETA: 2s - loss: 0.6090 - acc: 0.6707
Epoch 00002: val_loss improved from 0.64114 to 0.55401, saving model to models//5-band-CNN-correct-filters.hdf5
785/785 [==============================] - 2589s 3s/step - loss: 0.6090 - acc: 0.6707 - val_loss: 0.5540 - val_acc: 0.7623
Epoch 3/20
784/785 [============================>.] - ETA: 2s - loss: 0.5782 - acc: 0.6941
Epoch 00003: val_loss improved from 0.55401 to 0.52204, saving model to models//5-band-CNN-correct-filters.hdf5
785/785 [==============================] - 2579s 3s/step - loss: 0.5782 - acc: 0.6942 - val_loss: 0.5220 - val_acc: 0.8565
Epoch 4/20
784/785 [=================

In [31]:
#test_loss, test_acc = model.evaluate(test_images, test_target)
#print('Test accuracy:', test_acc)
#print('Test loss:', test_loss)

In [37]:
filepath = "models//5-band-CNN-correct-filters-little-more.hdf5"
ModelCheckpointCB = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

In [39]:
hist_1 = model.fit_generator(generator=training_generator,
                    steps_per_epoch=steps_to_take, 
                    epochs=25,
                    validation_data=validation_generator,
                    validation_steps=val_steps_to_take,
                    verbose=1,
                    initial_epoch=21,
                    callbacks=[ModelCheckpointCB,lr_callback])
                    #callbacks=[tensorboard_callback,lr_callback])

Epoch 22/25
784/785 [============================>.] - ETA: 2s - loss: 0.2327 - acc: 0.8889
Epoch 00022: val_loss improved from inf to 0.12059, saving model to models//5-band-CNN-correct-filters-little-more.hdf5
785/785 [==============================] - 2611s 3s/step - loss: 0.2325 - acc: 0.8890 - val_loss: 0.1206 - val_acc: 0.9574
Epoch 23/25
784/785 [============================>.] - ETA: 2s - loss: 0.2243 - acc: 0.8927
Epoch 00023: val_loss improved from 0.12059 to 0.10971, saving model to models//5-band-CNN-correct-filters-little-more.hdf5
785/785 [==============================] - 2575s 3s/step - loss: 0.2243 - acc: 0.8927 - val_loss: 0.1097 - val_acc: 0.9598
Epoch 24/25
784/785 [============================>.] - ETA: 2s - loss: 0.2218 - acc: 0.8949
Epoch 00024: val_loss did not improve from 0.10971
785/785 [==============================] - 2623s 3s/step - loss: 0.2219 - acc: 0.8949 - val_loss: 0.1215 - val_acc: 0.9553
Epoch 25/25
784/785 [============================>.] - ETA: 

In [32]:
#y_prob = model.predict(X)

In [33]:
#source list
"""
https://fizzylogic.nl/2017/05/08/monitor-progress-of-your-keras-based-neural-network-using-tensorboard/

https://arxiv.org/pdf/1711.05744.pdf

https://arxiv.org/pdf/1807.00807.pdf

https://github.com/jameslawlor/kaggle_galaxy_zoo/blob/master/galaxy_zoo_keras.ipynb

https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

#https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions/48097478

https://distill.pub/2018/building-blocks/ what I want to do with this after it is working.
"""

'\nhttps://fizzylogic.nl/2017/05/08/monitor-progress-of-your-keras-based-neural-network-using-tensorboard/\n\nhttps://arxiv.org/pdf/1711.05744.pdf\n\nhttps://arxiv.org/pdf/1807.00807.pdf\n\nhttps://github.com/jameslawlor/kaggle_galaxy_zoo/blob/master/galaxy_zoo_keras.ipynb\n\nhttps://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly\n\n#https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions/48097478\n\nhttps://distill.pub/2018/building-blocks/ what I want to do with this after it is working.\n'

In [34]:
"""
#model died, I need to implement a save model callback

785/785 [==============================] - 2908s 4s/step - loss: 1.3123 - acc: 0.7658 - val_loss: 0.3576 - val_acc: 0.8862
Epoch 2/15
785/785 [==============================] - 2804s 4s/step - loss: 0.2477 - acc: 0.8998 - val_loss: 0.2727 - val_acc: 0.9176
Epoch 3/15
785/785 [==============================] - 2730s 3s/step - loss: 0.2065 - acc: 0.9165 - val_loss: 0.2536 - val_acc: 0.9209
Epoch 4/15
785/785 [==============================] - 2701s 3s/step - loss: 0.1747 - acc: 0.9301 - val_loss: 0.3375 - val_acc: 0.8857
Epoch 5/15
574/785 [====================>.........] - ETA: 10:50 - loss: 0.1768 - acc: 0.9268
"""

"""
784/785 [============================>.] - ETA: 3s - loss: 1.6864 - acc: 0.5130
Epoch 00001: val_loss improved from inf to 0.68030, saving model to models//5-band-CNN
785/785 [==============================] - 3110s 4s/step - loss: 1.6851 - acc: 0.5130 - val_loss: 0.6803 - val_acc: 0.5537
Epoch 2/10
784/785 [============================>.] - ETA: 2s - loss: 0.6738 - acc: 0.5632
Epoch 00002: val_loss improved from 0.68030 to 0.65193, saving model to models//5-band-CNN
785/785 [==============================] - 2741s 3s/step - loss: 0.6738 - acc: 0.5632 - val_loss: 0.6519 - val_acc: 0.5730
Epoch 3/10
784/785 [============================>.] - ETA: 2s - loss: 0.6481 - acc: 0.5953
Epoch 00003: val_loss improved from 0.65193 to 0.65075, saving model to models//5-band-CNN
785/785 [==============================] - 2677s 3s/step - loss: 0.6480 - acc: 0.5953 - val_loss: 0.6507 - val_acc: 0.6143
Epoch 4/10
784/785 [============================>.] - ETA: 2s - loss: 0.6387 - acc: 0.6149
Epoch 00004: val_loss did not improve from 0.65075
785/785 [==============================] - 2680s 3s/step - loss: 0.6387 - acc: 0.6149 - val_loss: 0.7353 - val_acc: 0.6195
Epoch 5/10
784/785 [============================>.] - ETA: 2s - loss: 0.5721 - acc: 0.6687
Epoch 00005: val_loss improved from 0.65075 to 0.39564, saving model to models//5-band-CNN
785/785 [==============================] - 2673s 3s/step - loss: 0.5718 - acc: 0.6689 - val_loss: 0.3956 - val_acc: 0.8078
Epoch 6/10
784/785 [============================>.] - ETA: 2s - loss: 0.2843 - acc: 0.8756
Epoch 00006: val_loss did not improve from 0.39564
785/785 [==============================] - 2667s 3s/step - loss: 0.2842 - acc: 0.8756 - val_loss: 0.4010 - val_acc: 0.8995
Epoch 7/10
784/785 [============================>.] - ETA: 2s - loss: 0.2060 - acc: 0.9138
Epoch 00007: val_loss did not improve from 0.39564
785/785 [==============================] - 2677s 3s/step - loss: 0.2062 - acc: 0.9138 - val_loss: 0.4315 - val_acc: 0.8881
Epoch 8/10
784/785 [============================>.] - ETA: 2s - loss: 0.1737 - acc: 0.9292
Epoch 00008: val_loss improved from 0.39564 to 0.24017, saving model to models//5-band-CNN
785/785 [==============================] - 2675s 3s/step - loss: 0.1738 - acc: 0.9292 - val_loss: 0.2402 - val_acc: 0.9069
Epoch 9/10
784/785 [============================>.] - ETA: 2s - loss: 0.1621 - acc: 0.9383
Epoch 00009: val_loss did not improve from 0.24017
785/785 [==============================] - 2678s 3s/step - loss: 0.1620 - acc: 0.9383 - val_loss: 0.3230 - val_acc: 0.9297
Epoch 10/10
784/785 [============================>.] - ETA: 2s - loss: 0.1451 - acc: 0.9423
Epoch 00010: val_loss did not improve from 0.24017
785/785 [==============================] - 2667s 3s/step - loss: 0.1451 - acc: 0.9422 - val_loss: 0.2926 - val_acc: 0.9496

Intesting, it seems I am not very stable. I want to run this a few more epochs then throw on xception.

#after running for a few more epochs, I have decided to re-start. it seems after ~13 epochs the model loses all stability. 
#begins increasing dramatically. I am lowering starting LR and placing in a LR scheduler.
"""

'\n784/785 [============================>.] - ETA: 3s - loss: 1.6864 - acc: 0.5130\nEpoch 00001: val_loss improved from inf to 0.68030, saving model to models//5-band-CNN\n785/785 [==============================] - 3110s 4s/step - loss: 1.6851 - acc: 0.5130 - val_loss: 0.6803 - val_acc: 0.5537\nEpoch 2/10\n784/785 [============================>.] - ETA: 2s - loss: 0.6738 - acc: 0.5632\nEpoch 00002: val_loss improved from 0.68030 to 0.65193, saving model to models//5-band-CNN\n785/785 [==============================] - 2741s 3s/step - loss: 0.6738 - acc: 0.5632 - val_loss: 0.6519 - val_acc: 0.5730\nEpoch 3/10\n784/785 [============================>.] - ETA: 2s - loss: 0.6481 - acc: 0.5953\nEpoch 00003: val_loss improved from 0.65193 to 0.65075, saving model to models//5-band-CNN\n785/785 [==============================] - 2677s 3s/step - loss: 0.6480 - acc: 0.5953 - val_loss: 0.6507 - val_acc: 0.6143\nEpoch 4/10\n784/785 [============================>.] - ETA: 2s - loss: 0.6387 - acc: 0

In [35]:
#Here is how to load in a model's weights

#model_new = create_model()
#model_new.load_weights("models//5-band-CNN.h5")

In [26]:
#test model performance:
model.load_weights("models//5-band-CNN-correct-filters-little-more.hdf5")

hist2 = model.evaluate_generator(generator=test_generator,
                    steps=test_steps_to_take, 
                    verbose=1)


C:\Users\awe2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


241/241 [==============================] - 664s 3s/step - loss: 0.1155 - acc: 0.9593 - auc: 0.9947


In [16]:
print(type(hist2))

<class 'list'>
